<a href="https://colab.research.google.com/github/ilman79/Machine-Learning/blob/main/Development_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Akses untuk melihat sumber data dari website kaggle : [dataset](https://www.kaggle.com/datasets/ahemateja19bec1025/traffic-sign-dataset-classification?resource=download)

In [ ]:
#import data set


In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-171-g6e04b94 Python-3.10.11 torch-2.0.1+cu118 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 23.3/107.7 GB disk)


In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="lpNWV82uE9gZMd1p5IsI")
project = rf.workspace("ilman-gifari-utdmy").project("rrd-htsjl")
dataset = project.version(1).download("folder")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.0 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=29f0844514469e6c1951f31761171682ddfd09bac66971aac86bc81c406f0178
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.9
    Uninstalling pyparsing-3.0.9:
      Successfully uninstalled pyparsing-3.0.9
  Attempting uninstall: idna
    Found existing installation: idna 3.4
    Uninstalling idna-3.4:
    

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to RRD-1 in folder:: 100%|██████████| 480/480 [00:00<00:00, 556.16it/s]


In [2]:
%cd ../yolov5
from utils.downloads import attempt_download

p5 = ['n', 's', 'm', 'l', 'x']  # P5 models
cls = [f'{x}-cls' for x in p5]  # classification models

for x in cls:
    attempt_download(f'weights/yolov5{x}.pt')

/content/yolov5


100%|██████████| 4.87M/4.87M [00:00<00:00, 188MB/s]

100%|██████████| 10.5M/10.5M [00:00<00:00, 275MB/s]

100%|██████████| 24.9M/24.9M [00:00<00:00, 174MB/s]

100%|██████████| 50.9M/50.9M [00:00<00:00, 207MB/s]

100%|██████████| 92.0M/92.0M [00:00<00:00, 163MB/s]



In [6]:
!python classify/train.py --model yolov5s-cls.pt --data $DATASET_NAME --epochs 5 --img 163 --pretrained weights/yolov5s-cls.pt


usage: train.py
       [-h]
       [--model MODEL]
       [--data DATA]
       [--epochs EPOCHS]
       [--batch-size BATCH_SIZE]
       [--imgsz IMGSZ]
       [--nosave]
       [--cache [CACHE]]
       [--device DEVICE]
       [--workers WORKERS]
       [--project PROJECT]
       [--name NAME]
       [--exist-ok]
       [--pretrained [PRETRAINED]]
       [--optimizer {SGD,Adam,AdamW,RMSProp}]
       [--lr0 LR0]
       [--decay DECAY]
       [--label-smoothing LABEL_SMOOTHING]
       [--cutoff CUTOFF]
       [--dropout DROPOUT]
       [--verbose]
       [--seed SEED]
       [--local_rank LOCAL_RANK]
train.py: error: argument --data: expected one argument
